In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from camera import CameraReader, display_frame
import ipywidgets as widgets
import numpy as np
import sys
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import pywt
import pickle
from sklearn.metrics import mean_squared_error
import time
from scipy.cluster.vq import vq
from sklearn.cluster import KMeans
from matplotlib import animation, patches
from scipy import fftpack
from scipy.signal import fftconvolve, convolve2d
from IPython.display import display, Audio, HTML
from style import *
from collections import Counter,OrderedDict

%matplotlib notebook
plt.rcParams['image.cmap'] = 'gray'



In [3]:
def contar(matr):
    plano = np.ravel(matr.astype(np.uint8))
    return Counter(plano).most_common()

In [4]:
class Nodo:
    def __init__(self,hijo_izq,hijo_der):
        self.left = hijo_izq
        self.right = hijo_der
    def hijos(self):
        return (self.left,self.right)
    
def crear_arbol_de_Huffman(frecuencias):
    
    arbol_Hf = frecuencias
    while len(arbol_Hf)>1:        
        right,left = arbol_Hf[0],arbol_Hf[1]
        rm0x,rm0y = right[0],right[1]
        rm1x,rm1y = left[0],left[1]
        if(left[1]<right[1]):
            if(len(arbol_Hf)>2):
                aux1 = left
                aux2 = arbol_Hf[2]
                arbol_Hf[2] = right
                left = aux2
                right = aux1
            else:
                aux = left
                left = right
                right = aux
        nodo = Nodo(left,right)
        arbol_Hf.insert(0,(nodo,left[1]+right[1]))
        
        
        arbol_Hf.remove((rm0x,rm0y))
        arbol_Hf.remove((rm1x,rm1y))
    return arbol_Hf
        
def recorrer_arbol(nodo, prefijo='', codigo={}):
    if isinstance(nodo[0].left[0],Nodo):
        recorrer_arbol(nodo[0].left,prefijo+'1',codigo)
    else:
        codigo[str(nodo[0].left[0])]=prefijo+'1'
    if isinstance(nodo[0].right[0],Nodo):
        recorrer_arbol(nodo[0].right,prefijo+'0',codigo)
    else:
        codigo[str(nodo[0].right[0])]=prefijo+'0'
    
    return codigo
    

In [5]:
def replica(matrix,N=2):
    x = np.repeat(matrix[:,],N,axis=1)
    return np.repeat(x,N,axis=0)
def inversa(LL):
    LLrep = replica(LL)
    return (LLrep)

In [6]:
def Cuantizacion(x,h):
    return (x//h)*h + (h//2)

In [7]:
def LowRow(image):
    lowRowImage = np.zeros(shape=(image.shape[0],image.shape[1]//2))
    contt = 0
    for i in range(0,image.shape[1],2):        
        lowRowImage[:,contt:contt+1] = (image[:,i:i+1] + image[:,i+1:i+2])/2                
        contt += 1
    return np.clip(lowRowImage,1,250)

def LowCol(image):
    lowColImage = np.zeros(shape=(image.shape[0]//2,image.shape[1]))
    contt = 0
    for i in range(0,image.shape[0],2):
        lowColImage[contt:contt+1,:] = (image[i:i+1,:] + image[i+1:i+2,:])/2
        contt += 1
    return np.clip(lowColImage,1,250)

In [8]:
def compress(dic,content):
    res = ""
    for ch in content:
        code = dic[ch]
        res = res + code
    res = '1' + res + dic['end'] 
    res = res + (len(res) % 8 * '0')
    return int(res,2)


def store(k,dic,outn):
    ind=list(k.shape)
    res=[]
    outf = open(outn,'wb')
    for y in range(0,int(ind[1])):
        res=[]
        for x in range(0,int(ind[0])):
            res.append(str(int(k[x,y])))
        com=compress(dic,res)
        pickle.dump(com,outf)
    outf.close()
    
def decodif(dic, bitstr):
    res = []
    length = bitstr.bit_length() - 1
    if bitstr >> length != 1:
        raise Error("Corrupt file!")
    done = False
    while length > 0 and not done:
        shift = length - 1
        while True:
            num = bitstr >> shift
            bitnum = bin(num)[3:]
            if bitnum not in dic.values():
                shift -= 1
                continue
            char = list(dic.keys())[list(dic.values()).index(bitnum)]
            if char == 'end':
                done = True
                break
            res.append(char)
            bitstr = bitstr - ((num - 1) << shift)
            length = shift
    return res
    
#Lee el archivo .phuijseheise y llama a la funcion decodif para decodificar
def decode(ind,dic,outn):
    f = open(outn,'rb')
    objects = []
    while True:
            try:
                objects.append(pickle.load(f))
            except EOFError:
                break
    f.close()
    h = np.zeros((int(ind[0]),int(ind[1])))

    for y in range(0,int(ind[1])):
        jue = decodif(dic,objects[((int(ind[1])*0)+y)])
        cont = 0
        for x in range(0,int(ind[0])):
            h[x,y]=jue[cont]
            cont=cont+1
    return h

def limpiar_filtro(frame,size_y,size_x):
    #Se obtiene su espectro en frecuencias
    frame_limpio = fftpack.fftshift(fftpack.fft2(frame))
    #Se aplica un filtro al espectro
    frame_limpio[240-size_y-1-frequency:240+size_y-1-frequency, 420-size_x+2:420+size_x+2] = 0
    frame_limpio[240-size_y-1+frequency:240+size_y+1+frequency, 420-size_x+6:420+size_x+6] = 0
    #Se aplica inversa para volver a la imagen original
    frame_ret = np.abs(fftpack.ifft2(fftpack.ifftshift(frame_limpio)))
    return frame_ret


def play_vid(frame_list,size_y,size_x,out_n):
    count = 0
    display(out)
    for frame in cam:
        limpio = limpiar_filtro(frame,size_y,size_x)

        lowRow = LowRow(limpio)
        LL = LowCol(lowRow) #Primer nivel
        
        lowRow = LowRow(LL)
        LL2 = LowCol(lowRow) #Segundo nivel #descomentar hasta aqui si quieren 2do nivel
        
        #lowRow = LowRow(LL2)
        #LL3 = LowCol(lowRow) #Tercer nivel #descomentar hasta aqui si quieren 3er nivel
        
        #lowRow = LowRow(LL3)
        #LL4 = LowCol(lowRow) #Cuarto nivel #descomentar hasta aqui si quieren 4to nivel
        
        n = 4 #Cantidad de grises = 256//n 
        
        #imagen_wv = Cuantizacion(LL1,n)  #descomentar si quieren ocupar er nivel
        imagen_wv = Cuantizacion(LL2,n)  #descomentar si quieren ocupar 2er nivel
        #imagen_wv = Cuantizacion(LL3,n) #descomentar si quieren ocupar 3er nivel
        #imagen_wv = Cuantizacion(LL4,n) #descomentar si quieren ocupar 4to nivel
        
        freq_wv = contar(imagen_wv)
        freq_wv.append(('end',1))
        wv_tree = crear_arbol_de_Huffman(freq_wv[::-1])
        wv_dict = recorrer_arbol(wv_tree[0],'',{})

        store(imagen_wv,wv_dict,out_n)
            
        h=decode(list(imagen_wv.shape),wv_dict,out_n)
        
        #h = inversa(h) #descomentar si quieren 1er nivel
        h = inversa(inversa(h)) #descomentar si quieren 2do nivel
        #h = inversa(inversa(inversa(h))) #descomentar si quieren hacer 3er nivel
        #h = inversa(inversa(inversa(inversa(h)))) #descomentar si quieren ocupar 4to nivel
        
        #mse = mean_squared_error(h.astype(np.uint8), limpio.astype(np.uint8))
        #print(mse)

        
        count +=1;
        #if(count == 1):
        #    break
        display_frame(out, h.astype(np.uint8) )        
    return count    

def play_sp(out,cam):
    count = 0
    for i in cam:
        im = scipy.misc.imread("frame_sp/sp{0}.jpg".format(count))
        display_frame(out,im)
        count 

In [9]:
cam = CameraReader()
vid_height, vid_width = cam.get_resolution()
print("Resolución del video: {0}x{1}p".format(vid_width, vid_height))
print("Cuadros por segundo: {0}".format(cam.get_fps()))
print()
out = widgets.Image(layout=widgets.Layout(width=str(vid_width), 
                                           height=str(vid_height)))

size_y=10
size_x=2
frequency=28
frame_list = [[]]
time1 = time.time()
name_out = "codificacion.phuijseheise"
count = play_vid(frame_list,size_y,size_x,name_out)
time2 = time.time()
print("Total = ", time2 - time1, "segundos")
print(count/(time2-time1),"FPS")

Iniciando lector de camara
OpenCV version: major: 4, minor: 1, subminor: 0
Resolución del video: 848x480p
Cuadros por segundo: 29.97002997002997



Image(value=b'', layout="Layout(height='480', width='848')")

KeyboardInterrupt: 